In [1]:
import json
import pandas as pd
import geopandas as gpd

from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer

In [2]:
import folium

In [3]:
import branca.colormap as cm
from folium import plugins
from folium.plugins import MeasureControl
from folium.plugins import FloatImage

In [4]:
df = pd.read_csv("df_rodent_violations_zips_counts.csv")
df.head()

,ZIPCODE,violation_count
0,10003.0,777
1,10002.0,470
2,10016.0,436
3,10013.0,422
4,11220.0,413


In [5]:
df['ZIPCODE'] = df['ZIPCODE'].astype(int)

In [6]:
df['ZIPCODE'] = df['ZIPCODE'].astype(str)

In [7]:
zipfile = 'zipcode.geojson'

In [8]:
gdf = gdf = gpd.read_file(zipfile)

In [9]:
df.rename(index=str, columns={'ZIPCODE': 'postalCode'}, inplace=True)
df.head()

,postalCode,violation_count
0,10003,777
1,10002,470
2,10016,436
3,10013,422
4,11220,413


In [16]:
merged_json

{'type': 'FeatureCollection',
 'features': [{'id': '0',
   'type': 'Feature',
   'properties': {'OBJECTID': 1,
    'postalCode': '11372',
    'PO_NAME': 'Jackson Heights',
    'STATE': 'NY',
    'borough': 'Queens',
    'ST_FIPS': '36',
    'CTY_FIPS': '081',
    'BLDGpostalCode': 0,
    'Shape_Leng': 20624.6923165,
    'Shape_Area': 20163283.8744,
    '@id': 'http://nyc.pediacities.com/Resource/PostalCode/11372',
    'violation_count': 284.0},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-73.86942457284175, 40.74915687096787],
      [-73.89507143240856, 40.74646547081214],
      [-73.89618737867819, 40.74850942518086],
      [-73.89583954185139, 40.748546875706005],
      [-73.89525242774396, 40.74830660945023],
      [-73.89654041085561, 40.750541998143575],
      [-73.89579868613828, 40.750619721332605],
      [-73.89652230661433, 40.75438879610902],
      [-73.87221855882477, 40.756943248067465],
      [-73.87167992356791, 40.75398717439602],
      [-73.87207046513889, 

In [11]:
merge = gdf.merge(df, how='left', on='postalCode')
merge.head()

,OBJECTID,postalCode,PO_NAME,STATE,borough,ST_FIPS,CTY_FIPS,BLDGpostalCode,Shape_Leng,Shape_Area,@id,geometry,violation_count
0,1,11372,Jackson Heights,NY,Queens,36,081,0,20624.692317,2.016328e+07,http://nyc.pediacities.com/Resource/PostalCode...,POLYGON ((-73.86942457284175 40.74915687096787...,284.0
1,2,11004,Glen Oaks,NY,Queens,36,081,0,23002.816039,2.260653e+07,http://nyc.pediacities.com/Resource/PostalCode...,POLYGON ((-73.71068374074007 40.75004039450917...,12.0
2,3,11040,New Hyde Park,NY,Queens,36,081,0,15749.161511,6.269333e+06,http://nyc.pediacities.com/Resource/PostalCode...,POLYGON ((-73.70098278625547 40.73889569923034...,14.0
3,4,11426,Bellerose,NY,Queens,36,081,0,35932.810639,4.941836e+07,http://nyc.pediacities.com/Resource/PostalCode...,POLYGON ((-73.72270447144122 40.75373371438336...,27.0
4,5,11365,Fresh Meadows,NY,Queens,36,081,0,38693.565676,6.938587e+07,http://nyc.pediacities.com/Resource/PostalCode...,POLYGON ((-73.81088634744755 40.72717187575918...,32.0


In [12]:
merged_json = json.loads(merge.to_json())

In [13]:
json_data = json.dumps(merged_json)

In [14]:
json_data

'{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"OBJECTID": 1, "postalCode": "11372", "PO_NAME": "Jackson Heights", "STATE": "NY", "borough": "Queens", "ST_FIPS": "36", "CTY_FIPS": "081", "BLDGpostalCode": 0, "Shape_Leng": 20624.6923165, "Shape_Area": 20163283.8744, "@id": "http://nyc.pediacities.com/Resource/PostalCode/11372", "violation_count": 284.0}, "geometry": {"type": "Polygon", "coordinates": [[[-73.86942457284175, 40.74915687096787], [-73.89507143240856, 40.74646547081214], [-73.89618737867819, 40.74850942518086], [-73.89583954185139, 40.748546875706005], [-73.89525242774396, 40.74830660945023], [-73.89654041085561, 40.750541998143575], [-73.89579868613828, 40.750619721332605], [-73.89652230661433, 40.75438879610902], [-73.87221855882477, 40.756943248067465], [-73.87167992356791, 40.75398717439602], [-73.87207046513889, 40.75386200705204], [-73.86942457284175, 40.74915687096787]]]}}, {"id": "1", "type": "Feature", "properties": {"OBJEC

In [15]:
geosource = GeoJSONDataSource(geojson = json_data)

palette = brewer['PuRd'][8]
palette = palette[::-1]

color_mapper = LinearColorMapper(palette = palette, low = 0, high = 777,  nan_color = '#d9d9d9')



color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color='black',location = (0,0), orientation ='horizontal')#, major_label_overrides = tick_labels)

p = figure(title = 'Rat Violation Counts', plot_height = 700 , plot_width = 700, toolbar_location = None, 
          tooltips=[
         ("Name", "@PO_NAME"),("Zip Code","@postalCode"), ("Violation Count", "@violation_count")
    ])
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

p.patches('xs','ys', source = geosource,fill_color = {'field' :'violation_count', 'transform' : color_mapper},
         line_color = 'black', line_width = 0.25, fill_alpha = 1)


p.add_layout(color_bar, 'below')

output_notebook()

show(p)

Loading BokehJS ...

In [112]:
m = folium.Map(location=[40.7128, -74.0060], zoom_start=11, control_scale=True )

m.choropleth(
    geo_data=merged_json,
    name = 'Rodent Violation Count Per Zip Code',
    fill_color = 'PuRd',
    fill_opacity=0.6,
    line_opacity=0.8,
    data=df,
    legend_name = 'Rodent Violoation Count',
    key_on = 'feature.properties.postalCode',
    columns= ['postalCode', 'violation_count'], 
    highlight=True
      )

folium.LayerControl().add_to(m)


/anaconda3/envs/learn-env/lib/python3.6/site-packages/folium/folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [80]:
m